In [1]:
import sys
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from trade.stocks import Stocks
from trade.options import Options
from trade.utils import days_until
from trade.tradable_instrument import TradableInstrument

# Add the project root directory to the Python path
project_root = os.path.abspath('..')
sys.path.insert(0, project_root)

# Now you can import the constants module
from config import constants_short_put_credit_spreads as configs

In [2]:
tickers = configs.TICKERS
margin = configs.MARGIN

low_days_cutoff = 1
high_days_cutoff = 10

In [3]:
def find_put_spread(puts_f, last_f, margin=.1):

    short_put_candidates = puts_f[puts_f['strike'] <= last_f]
    put_spreads = []
    for _, short_put in short_put_candidates.iterrows():
        long_put_candidates = puts_f[puts_f['strike'] < short_put['strike']]
        if not long_put_candidates.empty:
            long_put = long_put_candidates.iloc[0]  # Get the first available long put

            # Calculate credit and risk
            short_price = (short_put['bid']*.5 + short_put['ask']*.5)
            long_price = (long_put['bid']*.5+long_put['ask']*.5)
            credit = short_price - long_price#short_put['bid'] - long_put['ask']
            market_credit = short_put['bid']-long_put['ask']
            risk = short_put['strike'] - long_put['strike'] - credit
            market_risk = short_put['strike'] - long_put['strike'] - market_credit

            # Check if credit is greater than 50% of the risk   
            if credit > margin * risk:
                put_spreads.append({
                    'short_put_contract': short_put['contractSymbol'],
                    'short_put_strike': short_put['strike'],
                    'short_bid':short_put['bid'],
                    'short_ask':short_put['ask'],
                    'short_price':short_price,
                    'long_put_contract': long_put['contractSymbol'],
                    'long_put_strike': long_put['strike'],
                    'long_bid':long_put['bid'],
                    'long_ask':long_put['ask'],
                    'long_price':long_price,
                    'last_price':last_f,
                    'credit': credit,
                    'market_credit':market_credit,
                    'risk': risk,
                    'market_risk':market_risk,
                    'c/r': credit/risk,
                    'market_c/r':market_credit/market_risk,
                    'ITM_return':(short_put['strike']-last_f)/last_f, 
                    'expected_return':''             
                })

    return pd.DataFrame(put_spreads)

In [11]:
"""Next we need to package the code in this loop"""

put_spreads_df = []
for ticker in tqdm(tickers):
    #try:
    if True:
        tradable_obj=TradableInstrument(ticker)
        option_chains = tradable_obj.options.get_option_chains_within_cutoff_days(
            low_days_cutoff=low_days_cutoff, high_days_cutoff=high_days_cutoff)
        #print(option_chains)
        if not option_chains:continue
        for e_d in list(option_chains.keys()):
            last, sma_10, sma_50 = tradable_obj.stock.get_current_stock_price()

            #puts = option_chains[e_d].puts
            #puts = puts[puts.inTheMoney==False]
            #puts=puts.sort_values(by='strike', ascending=False)
            puts = tradable_obj.options.get_otm_puts(expiration_date=e_d)
            put_spread = find_put_spread(puts_f=puts, last_f=last, margin=margin)

            
            if not put_spread.empty: # this should be moved into find put spread function
                days = days_until(date_str=e_d)  # this should be moved into find put spread function
                return_dist = tradable_obj.stock.get_historical_returns(interval=days) # this should be moved into find put spread function

                vals = return_dist[f'{days}-Day Return'].values
                p = np.zeros(put_spread.shape[0])
                for idx, itm_val in enumerate(put_spread['ITM_return']):
                    p[idx] = 1-len(vals[vals<itm_val])/len(vals)

                return_break_even = ((put_spread['short_put_strike']-put_spread['credit']) - put_spread['last_price'])/put_spread['last_price']
                p_break_even = np.zeros(put_spread.shape[0])
                for idx, bke_val in enumerate(return_break_even):
                    p_break_even[idx] = 1-len(vals[vals<bke_val])/len(vals)
                    
                put_spread['P(success)']=p
                put_spread['P(gain)']=p_break_even
                put_spread['P(loss)']=1-p_break_even
                put_spread['break_even_return'] = return_break_even
                put_spread["expected_value"] = p*put_spread['credit'] + (p_break_even-p)*put_spread['credit']/2 - (1-p_break_even)*put_spread['risk']
                put_spread["expected_return"] = (put_spread["expected_value"])/put_spread['risk']
                put_spread["expected_annual_return"] = (put_spread["expected_return"])/(days/365)
#
                put_spread["expected_market_value"] = p*put_spread['market_credit'] - (1-p)*put_spread['market_risk']
                put_spread["expected_market_return"] = (put_spread["expected_market_value"])/put_spread['market_risk']
                put_spread["expected_annual_market_return"] = (put_spread["expected_market_return"])/(days/365)
#
                put_spread['days']=days
                put_spread['sma_10']=sma_10
                put_spread['sma_50']=sma_50
                put_spreads_df.append(put_spread)
    #except:  continue    
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['FB']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
100%|██████████| 10/10 [00:05<00:00,  1.74it/s]


In [12]:
put_spreads_df

[    short_put_contract  short_put_strike  short_bid  short_ask  short_price  \
 0  AAPL241004P00227500             227.5       2.32       2.44        2.380   
 1  AAPL241004P00225000             225.0       1.42       1.49        1.455   
 2  AAPL241004P00222500             222.5       0.83       0.90        0.865   
 
      long_put_contract  long_put_strike  long_bid  long_ask  long_price  ...  \
 0  AAPL241004P00225000            225.0      1.42      1.49       1.455  ...   
 1  AAPL241004P00222500            222.5      0.83      0.90       0.865  ...   
 2  AAPL241004P00220000            220.0      0.50      0.54       0.520  ...   
 
     P(loss)  break_even_return  expected_value  expected_annual_return  \
 0  0.519698          -0.005334       -0.391498              -18.145614   
 1  0.418556          -0.014838       -0.464596              -17.756826   
 2  0.316289          -0.024738       -0.447913              -15.172931   
 
    expected_market_value  expected_market_return 